## Packages

In [1]:
import pandas as pd
import numpy as np
from math import ceil
from itertools import product
## import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import sqlalchemy 
from sqlalchemy import create_engine, text

import sys
import os

## Add the path of the functions folder
current_dir = os.getcwd()  ## Gets the current working directory
sub_dir = os.path.abspath(os.path.join(current_dir, '..'
                                       , 'Functions'))
sys.path.append(sub_dir)

# Now you can import functions
from db_secrets import SQL_107

from visualisations import plot_prediction_error, plot_prediction_density_subplots

from helpers import aggregate_sites, keras_calculate_accuracy, keras_calculate_baseline_accuracy

In [2]:
# TensorFlow sequential model
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Concatenate, LSTM, Dense, Dropout

from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam



from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [3]:
# Turn warnings off to keep notebook tidy
import warnings
warnings.filterwarnings("ignore")

## Connection

In [4]:
## text for query
with open("../Exploratory_Analysis/111_sql.sql", "r") as file:
    query_text = file.read()

query_text = query_text.replace('REPLACE START DATE','2022-01-01')

In [5]:
## Create an engine + connection
engine = create_engine(SQL_107())
conn = engine.connect()

## Return data
df_raw = pd.read_sql(query_text,conn)

## Wrangle

In [6]:
## Makes working copy
df = df_raw.copy()

#df = df.sample(n=100000, random_state=42)

In [7]:
## List columns
df.columns

Index(['Start_Location', 'Call ID', 'Pseudo NHS Number', 'CallDate',
       'Call Connect Time', 'Weekday_Name', 'Week_Start', 'Financial Year',
       'Bank Holiday', 'In_Out_Hours', 'Sub ICB Code', 'Sub ICB Name',
       'GP Practice', 'GP Practice Code', 'GP Deprivation',
       'GP Survey Q21 Wait for Appt', 'Symptom_Group',
       'Final Disposition Code', 'Disposition Group', 'Disposition',
       'Call_Taker_Triages', 'Clinical_Triages', 'Patient Age', 'Patient Sex',
       'Outcome ID', 'Outcome Datetime', 'Outcome Type', 'Outcome',
       'Outcome Location Code', 'Outcome Location Name', 'Hours to Outcome'],
      dtype='object')

In [8]:
df = df[['Call Connect Time'
         ,'Outcome Location Name'
         ,'Bank Holiday'
         , 'In_Out_Hours'
         , 'Sub ICB Name'
         ,'Outcome Type']].copy()

In [9]:
## Round time to nearest hour
df['Call Connect Time'] = df['Call Connect Time'].dt.round(freq='h')


In [10]:
## Replaces low frequency sites with 'OTHER SITE'
df['Outcome Location Name'] = (df['Outcome Location Name']
                               .apply(lambda x: aggregate_sites(x)))

In [11]:
df.head()

,Call Connect Time,Outcome Location Name,Bank Holiday,In_Out_Hours,Sub ICB Name,Outcome Type
0,2022-01-01 00:00:00,No UEC Contact,No,Out of Hours,Tees Valley,No UEC Contact
1,2022-01-01 00:00:00,No UEC Contact,No,Out of Hours,Tees Valley,No UEC Contact
2,2022-01-01 00:00:00,No UEC Contact,No,Out of Hours,Newcastle Gateshead,No UEC Contact
3,2022-01-01 01:00:00,No UEC Contact,No,Out of Hours,County Durham,No UEC Contact
4,2022-01-01 01:00:00,THE ROYAL VICTORIA INFIRMARY,No,Out of Hours,Northumberland,Emergency Care


#### binary outcome

In [12]:
df['Calls'] = 1

In [13]:
df['Outcome'] = df['Outcome Type'].transform(lambda x: 0 if x == 'No UEC Contact' else 1)
df = df.drop(['Outcome Type'],axis=1) 

# Reassemble data

#### ICB values

In [14]:
## Aggregates to one column per place per timestamp
df_call = pd.pivot_table(df
                        ,values = 'Calls'
                        ,index = 'Call Connect Time'
                        ,columns ='Sub ICB Name'
                        ,aggfunc ='sum'
                        ,fill_value = 0)

In [15]:
df_call.head()

Sub ICB Name,County Durham,Newcastle Gateshead,North Tyneside,Northumberland,South Tyneside,Sunderland,Tees Valley
Call Connect Time,,,,,,,
2022-01-01 00:00:00,3,4,1,4,0,1,6
2022-01-01 01:00:00,8,5,0,5,1,3,2
2022-01-01 02:00:00,4,3,5,5,1,2,2
2022-01-01 03:00:00,3,4,3,4,1,0,7
2022-01-01 04:00:00,2,2,1,3,0,0,5


#### Site values

In [16]:
df_site = df[df['Outcome']==1]

df_site = df_site[[ 'Call Connect Time'
         , 'Outcome Location Name'
         , 'Outcome'
         ,]].groupby([ pd.Grouper(key='Call Connect Time', freq='1h')
         , 'Outcome Location Name']).agg('sum').reset_index()

In [17]:
## Removes OTHER SITE
df_site = df_site[~(df_site['Outcome Location Name']=='OTHER SITE')]

In [18]:
df_site.head()

,Call Connect Time,Outcome Location Name,Outcome
0,2022-01-01,NORTHUMBRIA SPECIALIST EMERGENCY CARE HOSPITAL,1
1,2022-01-01,QUEEN ELIZABETH HOSPITAL,2
2,2022-01-01,SUNDERLAND ROYAL HOSPITAL,1
3,2022-01-01,UNIVERSITY HOSPITAL OF NORTH DURHAM,1
4,2022-01-01,UNIVERSITY HOSPITAL OF NORTH TEES,1


### Extra time features

In [19]:
df_times = df[['Call Connect Time'
            ,'Bank Holiday'
            , 'In_Out_Hours']].drop_duplicates()

In [20]:
df_times.head()

,Call Connect Time,Bank Holiday,In_Out_Hours
0,2022-01-01 00:00:00,No,Out of Hours
3,2022-01-01 01:00:00,No,Out of Hours
5,2022-01-01 02:00:00,No,Out of Hours
11,2022-01-01 03:00:00,No,Out of Hours
17,2022-01-01 04:00:00,No,Out of Hours


### Assemble

In [21]:
# Get unique timestamps and sites
unique_timestamps = df['Call Connect Time'].unique()
unique_sites = df_site['Outcome Location Name'].unique()

# Create a complete cross join of every site with every timestamp
complete_index = pd.DataFrame(product(unique_timestamps, unique_sites)
                              , columns=['Call Connect Time'
                                         , 'Outcome Location Name'])


In [22]:
## Merge time features
df = complete_index.merge(df_times, on='Call Connect Time', how='left')

In [23]:
## Merge calls + places
df = df.merge(df_call,on='Call Connect Time', how='left').fillna(0)

In [24]:
## Merge sites
df = df.merge(df_site,on=['Call Connect Time'
                           , 'Outcome Location Name'], how='left').fillna(0)

In [25]:
df.head()

,Call Connect Time,Outcome Location Name,Bank Holiday,In_Out_Hours,County Durham,Newcastle Gateshead,North Tyneside,Northumberland,South Tyneside,Sunderland,Tees Valley,Outcome
0,2022-01-01,NORTHUMBRIA SPECIALIST EMERGENCY CARE HOSPITAL,No,Out of Hours,3,4,1,4,0,1,6,1.0
1,2022-01-01,QUEEN ELIZABETH HOSPITAL,No,Out of Hours,3,4,1,4,0,1,6,2.0
2,2022-01-01,SUNDERLAND ROYAL HOSPITAL,No,Out of Hours,3,4,1,4,0,1,6,1.0
3,2022-01-01,UNIVERSITY HOSPITAL OF NORTH DURHAM,No,Out of Hours,3,4,1,4,0,1,6,1.0
4,2022-01-01,UNIVERSITY HOSPITAL OF NORTH TEES,No,Out of Hours,3,4,1,4,0,1,6,1.0


### date time

In [26]:
## Date time conversion to numeric
df['year']    = df['Call Connect Time'].dt.year

df['month sin'] = np.sin(df['Call Connect Time'].dt.month * (2*np.pi/12))
df['month cos'] = np.cos(df['Call Connect Time'].dt.month * (2*np.pi/12))

df['YearDay sin'] = np.sin(df['Call Connect Time'].dt.day_of_year * (2*np.pi/365))
df['YearDay cos'] = np.cos(df['Call Connect Time'].dt.day_of_year * (2*np.pi/365))

df['weekday sin'] = np.sin(df['Call Connect Time'].dt.weekday+1 * (2*np.pi/7))  # Monday=0, Sunday=6
df['weekday cos'] = np.cos(df['Call Connect Time'].dt.weekday+1 * (2*np.pi/7))  # Monday=0, Sunday=6

df['Hour sin'] = np.sin(df['Call Connect Time'].dt.hour * (2*np.pi/24))
df['Hour cos'] = np.cos(df['Call Connect Time'].dt.hour * (2*np.pi/24))

#df = df.drop('Call Connect Time',axis=1) 

df = df.set_index('Call Connect Time')


In [27]:
## One hot encodinng for boolean variables
bool_mapping = {
    'Yes': 1,
    'No': 0,
    'In Hours': 1,
    'Out of Hours': 0
}

df.loc[:,'Is Bank Holiday'] = df['Bank Holiday'].map(bool_mapping)             
df.loc[:,'In Hours'] = df['In_Out_Hours'].map(bool_mapping)
df = df.drop(['Bank Holiday','In_Out_Hours'],axis=1) 

In [28]:
## Dummy variables from Outcome Location Name	
#df = pd.concat([df, pd.get_dummies(df['Outcome Location Name']
#                                   ,dtype=int
#                                   , prefix='Site')]
#                ,axis=1)
#df = df.drop('Outcome Location Name', axis=1)

In [29]:
df.head()

,Outcome Location Name,County Durham,Newcastle Gateshead,North Tyneside,Northumberland,South Tyneside,Sunderland,Tees Valley,Outcome,year,month sin,month cos,YearDay sin,YearDay cos,weekday sin,weekday cos,Hour sin,Hour cos,Is Bank Holiday,In Hours
Call Connect Time,,,,,,,,,,,,,,,,,,,,
2022-01-01,NORTHUMBRIA SPECIALIST EMERGENCY CARE HOSPITAL,3,4,1,4,0,1,6,1.0,2022,0.5,0.866025,0.017213,0.999852,-0.376103,0.926578,0.0,1.0,0,0
2022-01-01,QUEEN ELIZABETH HOSPITAL,3,4,1,4,0,1,6,2.0,2022,0.5,0.866025,0.017213,0.999852,-0.376103,0.926578,0.0,1.0,0,0
2022-01-01,SUNDERLAND ROYAL HOSPITAL,3,4,1,4,0,1,6,1.0,2022,0.5,0.866025,0.017213,0.999852,-0.376103,0.926578,0.0,1.0,0,0
2022-01-01,UNIVERSITY HOSPITAL OF NORTH DURHAM,3,4,1,4,0,1,6,1.0,2022,0.5,0.866025,0.017213,0.999852,-0.376103,0.926578,0.0,1.0,0,0
2022-01-01,UNIVERSITY HOSPITAL OF NORTH TEES,3,4,1,4,0,1,6,1.0,2022,0.5,0.866025,0.017213,0.999852,-0.376103,0.926578,0.0,1.0,0,0


In [30]:
df.columns

Index(['Outcome Location Name', 'County Durham', 'Newcastle Gateshead',
       'North Tyneside', 'Northumberland', 'South Tyneside', 'Sunderland',
       'Tees Valley', 'Outcome', 'year', 'month sin', 'month cos',
       'YearDay sin', 'YearDay cos', 'weekday sin', 'weekday cos', 'Hour sin',
       'Hour cos', 'Is Bank Holiday', 'In Hours'],
      dtype='object')

## Build a baseline mode

Baseline split

In [31]:
base_X = df.drop('Outcome',axis=1)# X = all  except the 'Outcome' column
base_y = df['Outcome']# y = 'Outcome' column 

base_X_train, base_X_test, base_y_train, base_y_test = train_test_split(base_X
                                                    , base_y 
                                                    , test_size = 0.25
                                                    , random_state=42)

In [32]:
## Joins outcome onto predictors
base_df = pd.concat([base_X_train,base_y_train],axis=1)

In [33]:
## Columns we care about for baseline model
group_cols = [
    'month sin'
    ,'month cos'    
    ,'weekday sin'
    ,'weekday cos'
    ,'Hour sin'
    ,'Hour cos'] + df.columns[df.columns.str.startswith('Site_')].to_list() ## sites

## Mean value across baseline
base_trained = (base_df[group_cols + ['Outcome']]
                .groupby(group_cols)
                .agg( Pred_Outcome=pd.NamedAgg(column="Outcome"
                                               , aggfunc="mean"))
                )

In [34]:
base_y_pred_train = pd.merge(base_X_train,base_trained,how='left',on=group_cols)['Pred_Outcome']
base_y_pred_test = pd.merge(base_X_test,base_trained,how='left',on=group_cols)['Pred_Outcome']

### Lagged Features

In [35]:
def lag_data(df,col,one_hot=False,lags=[1,2,3,4,12,24,168]):
    """calcultes the lags for a dataframe column"""
    
    print(f' > Calculating lags for: {col}')
    
    lagged_features = pd.DataFrame()

    for x in lags:
        # Create lag features
        lagged_features[f'{col}_Lag{x}'] = df[col].shift(x)  # Previous x
        if one_hot & (x > 1):
            # Calculate rolling statistics
            lagged_features[f'{col}_mean{x}'] = df[col].rolling(window=x).mean()
            lagged_features[f'{col}_STD{x}']  = df[col].rolling(window=x).std() 
            lagged_features[f'{col}_min{x}']  = df[col].rolling(window=x).min() 
            lagged_features[f'{col}_max{x}']  = df[col].rolling(window=x).max() 
            lagged_features[f'{col}_median{x}']  = df[col].rolling(window=x).median()
            lagged_features[f'{col}_var{x}']  = df[col].rolling(window=x).var()

    df = pd.concat([df, lagged_features], axis=1)
    
    return(df)

In [36]:
cols_to_lag = ['Outcome']
cols_to_lag.extend(list(df_call.columns))

for col in cols_to_lag:
    df = lag_data(df,col,one_hot=True)

one_hot_cols_to_lag = ['Is Bank Holiday', 'In Hours']

for col in one_hot_cols_to_lag:
    df = lag_data(df,col,one_hot=False)

df = df[168:]

 > Calculating lags for: Outcome
 > Calculating lags for: County Durham
 > Calculating lags for: Newcastle Gateshead
 > Calculating lags for: North Tyneside
 > Calculating lags for: Northumberland
 > Calculating lags for: South Tyneside
 > Calculating lags for: Sunderland
 > Calculating lags for: Tees Valley
 > Calculating lags for: Is Bank Holiday
 > Calculating lags for: In Hours


### Outcome Location Name

In [37]:
## Dummy variables from Outcome Location Name	
df = pd.concat([df, pd.get_dummies(df['Outcome Location Name']
                                   ,dtype=int
                                   , prefix='Site')]
                ,axis=1)
df = df.drop('Outcome Location Name', axis=1)

## Time Series Data

In [38]:
#df = df[df['Outcome Location Name'] == 'NORTHUMBRIA SPECIALIST EMERGENCY CARE HOSPITAL']
#df = df.drop('Outcome Location Name',axis=1)
columns_order = (['Outcome'] + 
                 #['Outcome Location Name'] + 
                 [col for col in df.columns if col not in  ['Outcome','Outcome Location Name']] )
df = df[columns_order]
df.sort_index(inplace=True)

In [39]:
# Train-test split
test_train_split = 0.70
train_size = int(len(df) * test_train_split)

train_df = df[:train_size]
test_df = df[train_size:]

In [40]:
def scale_data(train, test):
    """Scale data 0-1 based on min and max in training set
        , excluding the first and second column"""
    
    # Initialise a new scaling object for normalising input data
    sc = MinMaxScaler()
    
    # Select columns to scale
    train_to_scale = train.iloc[:, 2:]
    test_to_scale = test.iloc[:, 2:]
    
    # Apply the scaler to the selected columns
    train_sc = sc.fit_transform(train_to_scale)
    test_sc = sc.transform(test_to_scale)
    
    # Combine the unscaled first column with the scaled remaining columns
    train_result = pd.concat([
        train.iloc[:, :2],  # Unscaled columns
        pd.DataFrame(train_sc, columns=train_to_scale.columns
                     , index=train.index)  # Scaled columns
    ], axis=1)
    
    test_result = pd.concat([
        test.iloc[:, :2],  # Unscaled columns
        pd.DataFrame(test_sc, columns=test_to_scale.columns
                     , index=test.index)  # Scaled columns
    ], axis=1)
    
    return train_result, test_result, sc

In [41]:
## Scale X data
train_df_sc, test_df_sc,scaler  = scale_data(train_df, test_df)

In [42]:
train_df_sc


,Outcome,County Durham,Newcastle Gateshead,North Tyneside,Northumberland,South Tyneside,Sunderland,Tees Valley,year,month sin,...,Site_QUEEN ELIZABETH HOSPITAL,Site_REDCAR PRIMARY CARE HOSPITAL,Site_SOUTH TYNESIDE DISTRICT HOSPITAL,Site_SUNDERLAND ROYAL HOSPITAL,Site_THE JAMES COOK UNIVERSITY HOSPITAL,Site_THE ROYAL VICTORIA INFIRMARY,Site_UNIVERSITY HOSPITAL OF HARTLEPOOL,Site_UNIVERSITY HOSPITAL OF NORTH DURHAM,Site_UNIVERSITY HOSPITAL OF NORTH TEES,Site_WANSBECK HOSPITAL
Call Connect Time,,,,,,,,,,,,,,,,,,,,,
2022-01-01 05:00:00,0.0,4,0.028571,0.023256,0.018868,0.000000,0.05,0.031008,0.0,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 05:00:00,0.0,4,0.028571,0.023256,0.018868,0.000000,0.05,0.031008,0.0,0.75,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 05:00:00,0.0,4,0.028571,0.023256,0.018868,0.000000,0.05,0.031008,0.0,0.75,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-01-01 05:00:00,0.0,4,0.028571,0.023256,0.018868,0.000000,0.05,0.031008,0.0,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2022-01-01 05:00:00,0.0,4,0.028571,0.023256,0.018868,0.000000,0.05,0.031008,0.0,0.75,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-16 19:00:00,2.0,38,0.414286,0.348837,0.226415,0.310345,0.30,0.325581,1.0,0.75,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2024-01-16 19:00:00,4.0,38,0.414286,0.348837,0.226415,0.310345,0.30,0.325581,1.0,0.75,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-01-16 19:00:00,2.0,38,0.414286,0.348837,0.226415,0.310345,0.30,0.325581,1.0,0.75,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:

#series_ids = df['Outcome Location Name'].values
#unique_series = df['Outcome Location Name'].unique()
#series_map = {s: i for i, s in enumerate(unique_series)}
#series_ids_encoded = np.array([series_map[s] for s in series_ids])

#series_one_hot = to_categorical(series_ids_encoded
#                                    , num_classes=len(unique_series))

#t = df.iloc[:,list(range(1,df.shape[1]))].values
#t = np.hstack([t, series_one_hot])

In [44]:
def make_windowed(df                 
                  ,window_length = 24*8
                  ,batch_size = 256
                  #,series_col = 'Outcome Location Name'
                  ):

    """Make data into windowed tensor for timeseries prediction"""

    ## Separate the series ID
    #series_ids = df[series_col].values
    
    ## One-hot encode series IDs
    #unique_series = df[series_col].unique()
    #series_map = {s: i for i, s in enumerate(unique_series)}
    #series_ids_encoded = np.array([series_map[s] for s in series_ids])
    #series_one_hot = to_categorical(series_ids_encoded
    #                                , num_classes=len(unique_series))

    ## Separate features (X) and targets (y)
    X = df.iloc[:,1:df.shape[1] ].values
    #X = np.hstack([X, series_one_hot])
    y = df.iloc[:,[0]]

    ## makes data into windowed tensors 
    dataset = timeseries_dataset_from_array(
        data=X,
        targets=y,
        sequence_length=window_length, 
        sampling_rate=1, #skip items
        batch_size=batch_size, #group together sequences efficiently in memory
    )

    return dataset #, len(unique_series) 

In [45]:
train_df_sc.shape[1]

389

In [47]:
## Sets derviation window length
window_length = 24*8 # 8 days

## Sets memory batch size
batch_size = 256

## makes data into windowed tensors 
#train_dataset, train_sites_N
train_dataset   = make_windowed(train_df_sc
                                             ,window_length
                                             ,batch_size)
                                             #,series_col = 'Outcome Location Name')

#test_dataset, test_sites_N
test_dataset    = make_windowed(test_df_sc
                                            ,window_length
                                            ,batch_size)
                                            #,series_col = 'Outcome Location Name')

In [48]:
print(train_dataset.batch(32)._structure[0])
print(train_dataset.batch(32)._structure[1])


TensorSpec(shape=(None, None, None, 388), dtype=tf.float64, name=None)
TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None)


In [53]:
def make_net(window_length,
            number_features, 
            #number_series,
            hidden_layers=1, 
            lstm_units=64,
            hidden_layer_neurones=128, 
            dropout=0.0, 
            learning_rate=0.003):
    
    """Make TensorFlow neural net"""
    
    ## Clear Tensorflow 
    K.clear_session()
    
    ## Set up neural net
    net = Sequential()

    ## input shape
    #net.add(InputLayer((window_length,number_features + number_series)))
    net.add(InputLayer((window_length,number_features)))

    ## LSTM
    net.add(LSTM(lstm_units,return_sequences = True))
    net.add(LSTM(int(lstm_units/2)))

    ## Add hidden hidden_layers using a loop
    for i in range(hidden_layers):
        # Add fully connected layer with ReLu activation
        net.add(Dense(
            hidden_layer_neurones, 
            input_dim=number_features,
            activation='relu'))
        # Add droput layer
        net.add(Dropout(dropout))
    
    ## Add linear activation output
    net.add(Dense(1, activation='linear'))  
    #net.add(Dense(number_series, activation='linear'))  

    #    net.add(Dense(1, activation='sigmoid'))    

    ## Set optimiser
    opt = Adam(learning_rate=learning_rate)
    
    ## Compiling model
    net.compile(loss='mse', 
                optimizer=opt, 
                metrics=['mse','mae'])
    
    return net

In [54]:
### Gets the number of features in the model
for X_batch, y_batch in train_dataset.take(1): #takes 1 batch
    print(f'X {X_batch.shape} | batch size={X_batch.shape[0]}, sequence length={X_batch.shape[1]}, features={X_batch.shape[2]}')
    print(f'y {y_batch.shape} | batch size={y_batch.shape[0]}, outcomes={y_batch.shape[1]}')

    number_features = X_batch.shape[2]

X (256, 192, 388) | batch size=256, sequence length=192, features=388
y (256, 1) | batch size=256, outcomes=1


In [55]:
model = make_net(window_length = window_length,
             number_features=number_features, 
             #number_series=1,
             lstm_units = 128,
             hidden_layers=1, 
             hidden_layer_neurones=128, 
             dropout=0.01, 
             learning_rate=0.0001)

In [56]:
### Summarise the model
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 192, 128)       │       264,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 322,561 (1.23 MB)

 Trainable params: 322,561 (1.23 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Define save checkpoint callback (only save if new best validation results)
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.keras', save_best_only=True)

# Define early stopping callback
# Stop when no validation improvement for 25 epochs
# Restore weights to best validation accuracy
early_stopping_cb_loss = keras.callbacks.EarlyStopping(
    patience=25, restore_best_weights=True, monitor='val_loss')

### Train model (and store training info in history)
history = model.fit(train_dataset,
                    epochs=10,
                    batch_size=batch_size,
                    validation_data=test_dataset,
                    verbose=1,
                    callbacks=[checkpoint_cb
                               , early_stopping_cb_loss
                               ])

Epoch 1/10
219/887 ━━━━━━━━━━━━━━━━━━━━ 10:06 907ms/step - loss: 4.1298 - mae: 1.4703 - mse: 4.1298

In [ ]:
### Function to plot loss 
def visualise_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


visualise_loss(history, "Training and Validation Loss")

In [ ]:
## get predictions
test_predictions = model.predict(test_dataset).flatten()
test_predictions = pd.DataFrame(data={'Predict':test_predictions})

## get actuals
test_actuals = test_df_sc.iloc[:len(test_predictions), [0]]
test_actuals.sort_index(inplace=True)
test_actuals['Mean']  = test_actuals['Outcome'].mean()

## combine into dataframe
test_predictions = pd.concat([
        test_actuals, 
        test_predictions.set_index(test_actuals.index)
    ], axis=1)

In [ ]:
## plot a sample of actual against predicted
minplot = 1000
maxplot = 1100

plt.figure(figsize=(10, 6))
plt.plot(test_predictions['Outcome'][minplot:maxplot], label="Actual")
plt.plot(test_predictions['Predict'][minplot:maxplot], label="Prediction")
plt.plot(test_predictions['Mean'][minplot:maxplot], label="Mean Actual")

# Add legend
plt.legend()

# Add title and axis labels for context
plt.title("TensorFlow LSTM: Predictions vs Actuals")
plt.xlabel("Time")
plt.ylabel("Value")

plt.show()


In [ ]:
def show_plot(plot_data, future, title):
    labels = ["History", "True Future", "Model Prediction"]
    marker = [".-", "rx", "go"]

    time_steps = list(range(-(plot_data[0].shape[0]), 0))
    future = list(range(future_steps))

    plt.title(title)
    for i, val in enumerate(plot_data):
        if i == 0:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
        else:            
            plt.plot(future, plot_data[i].flatten(), marker[i], label=labels[i])

    plt.legend()
    #plt.xlim([time_steps[0], (future + 5) * 2])
    plt.xlabel("Time-Step")
    plt.show()
    return

future_steps = 24
for x, y in test_dataset.take(5):
    show_plot(
        #[x[0][:, 1].numpy(), y[0].numpy(), model.predict(x)[0]],
        #1,
        [x[0][:, 1].numpy(),
          y[0].numpy()[:future_steps],
            model.predict(x)[0][:future_steps]],
        future_steps,
        "Single Step Prediction",
    )


# LEGACY BEYOND HERE

In [ ]:
xxxxx STOP xxxxx

In [39]:
def calculate_site_accuracy(df,model, X_train, X_test, y_train, y_test):
    """Calculate and print accuracy at site level of training and test data fits"""    
    
    X_df = df.drop('Outcome',axis=1)
    site_columns = X_df.columns[X_df.columns.str.startswith('Site_')]

    results = []

    for site in site_columns:
        
        # Get the column index for the site
        site_idx = X_df.columns.get_loc(site)

        # Filter array where site column equals 1
        test_index = np.where(X_test[:, site_idx] == 1)[0]  
        site_X_test = X_test[test_index]
        site_y_test = y_test[test_index]

        train_index = np.where(X_train[:, site_idx] == 1)[0]  
        site_X_train = X_train[train_index]
        site_y_train = y_train[train_index]

        # Predict on training and test data
        print(f'{site}: col {site_idx}')
        site_y_pred_train = model.predict(site_X_train).flatten()
        site_y_pred_test = model.predict(site_X_test).flatten()
    
        # Calculate Mean Absolute Error (MAE) for training and test sets
        site_mae_train = np.mean(np.abs(site_y_pred_train - site_y_train))
        site_mae_test = np.mean(np.abs(site_y_pred_test - site_y_test))
        
        # Calculate Mean Squared Error (MSE) for training and test sets
        site_mse_train = np.mean((site_y_pred_train - site_y_train) ** 2)
        site_mse_test = np.mean((site_y_pred_test - site_y_test) ** 2)

        # Calculate Root Mean Squared Error (RMSE) for training and test sets
        site_rmse_train = np.sqrt(site_mse_train)
        site_rmse_test = np.sqrt(site_mse_test)

        # Calculate NRMSE (Normalized RMSE)
        range_y_train = np.max(site_y_train) - np.min(site_y_train)  # Range of y_train
        range_y_test = np.max(site_y_test) - np.min(site_y_test)  # Range of y_test
        site_nrmse_train = site_rmse_train / range_y_train
        site_nrmse_test = site_rmse_test / range_y_test

        # Calculate R^2 for training and test sets
        ss_total_train = np.sum((site_y_train - np.mean(site_y_train)) ** 2)
        ss_total_test = np.sum((site_y_test - np.mean(site_y_test)) ** 2)
        ss_residual_train = np.sum((site_y_pred_train - site_y_train) ** 2)
        ss_residual_test = np.sum((site_y_pred_test - site_y_test) ** 2)

        r2_train = 1 - (ss_residual_train / ss_total_train)
        r2_test = 1 - (ss_residual_test / ss_total_test)

        ## results
        site_result = {'Site':site
                    ,'MAE train':site_mae_train                   
                    ,'MAE test':site_mae_test
                    ,'MSE train':site_mse_train
                    ,'MSE test':site_mse_test
                    ,'NRMSE train':site_nrmse_train
                    ,'NRMSE test':site_nrmse_test
                    ,'r2 train':r2_train
                    ,'r2 test':r2_test
                    }

        results.append(site_result)

    results = pd.DataFrame(results)

    return results

In [40]:
def plot_training(history_dict,measure='mae'):
    acc_values = history_dict[measure]
    val_acc_values = history_dict[f'val_{measure}']
    epochs = range(1, len(acc_values) + 1)

    fig, ax = plt.subplots()

    ax.set_xlabel('Time')
    ax.set_ylabel(measure)

    ax.plot(epochs, acc_values, color='blue', label=f'Training {measure}')
    ax.plot(epochs, val_acc_values, color='red', label=f'Test {measure}')
    ax.set_title(f'Training and validation {measure}')
    
    ax.legend()

    fig.show()

## Accuracy

In [ ]:
## Baseline accuracy is the mean of site, month, weekday, hour
keras_calculate_baseline_accuracy(base_y_pred_train
                                ,base_y_pred_test
                                ,base_y_train
                                ,base_y_test)


In [ ]:
keras_calculate_accuracy(model, X_train, X_test, y_train, y_test)

In [ ]:
site_results = calculate_site_accuracy(df,model
                                       , X_train
                                       , X_test
                                       , y_train
                                       , y_test)

In [ ]:
site_results

In [ ]:
plot_training(history.history,measure='mae')

In [ ]:
# Generate predictions
y_pred_train = model.predict(X_train).flatten()
y_pred_test = model.predict(X_test).flatten()

# Plot errors for both training and test data
plot_prediction_error(y_train, y_pred_train, title='Training Data - Prediction Error Plot')
plot_prediction_error(y_test, y_pred_test, title='Test Data - Prediction Error Plot')

In [ ]:
# Generate predictions
y_pred_train = model.predict(X_train).flatten()
y_pred_test = model.predict(X_test).flatten()

# Plot the density plots as subplots
plot_prediction_density_subplots(y_train, y_pred_train, y_test, y_pred_test)


In [ ]:
plot_prediction_density_subplots(base_y_train
                                 , base_y_pred_train
                                 , base_y_test
                                 , base_y_pred_test)